# Solar AI Challenge
***
### Objective
- Given weather data at 1-min frequency for 360 mins, predict the cloud coverage at 30 mins, 60 mins, 90 mins and 120 mins.
***
### Approach 1 
- Feed in the  360-min data into a LSTM as 1-min sequence.
- Pass the final hidden state into a decoder LSTM prediciting the weather data as a sequence for next 2 hours at 10-min intervals
- Use MSE of the 12 predicted sequences as *Loss criterion*
***

## Import Modules

In [ ]:
import os
import glob
from tqdm import trange
import pyprind
import time

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchsummary as summary

In [ ]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import random_split

In [ ]:
from sklearn.preprocessing import MinMaxScaler

## Helper Functions

In [ ]:
def get_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)
    
def scale_df(df):
    x = df.values
    scaler = MinMaxScaler()
    x_scaled = scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled, columns = df.columns)
    return df

In [ ]:
'''
def construct_fnn(input_size, layers):
    
    layer_list = []
    n_in = input_size
    p=0.2
    
    for i in layers:
        
        if i == 'D' :
            layer_list.append(nn.Dropout(p))
        elif i == 'R' :
            layer_list.append(nn.ReLU())
            layer_list.BatchNorm1d(i_last)
        elif i == 'T' :
            layer_list.append(nn.Tanh())
        elif i == 'S' :  
            layer_list.append(nn.Sigmoid())
        else :    
            layer_list.append(nn.Linear(n_in, i))
            i_last = i
            n_in = i
        
    return layer_list
'''
#nn.Sequential(*construct_fnn(10,[100,'T','D',200,'T','D',100,'T',1,'S']))

"\ndef construct_fnn(input_size, layers):\n    \n    layer_list = []\n    n_in = input_size\n    p=0.2\n    \n    for i in layers:\n        \n        if i == 'D' :\n            layer_list.append(nn.Dropout(p))\n        elif i == 'R' :\n            layer_list.append(nn.ReLU())\n            layer_list.BatchNorm1d(i_last)\n        elif i == 'T' :\n            layer_list.append(nn.Tanh())\n        elif i == 'S' :  \n            layer_list.append(nn.Sigmoid())\n        else :    \n            layer_list.append(nn.Linear(n_in, i))\n            i_last = i\n            n_in = i\n        \n    return layer_list\n"

In [ ]:
def construct_fnn(input_size, output_size, layers):
    
    layer_list = []
    n_in = input_size
    n_out = output_size
    p=0.2
    
    for i in layers:
        layer_list.append(nn.Linear(n_in, i))
        layer_list.append(nn.Tanh())
        layer_list.append(nn.BatchNorm1d(i))
        layer_list.append(nn.Dropout(p))
        n_in = i
        
    layer_list.append(nn.Linear(layers[-1], n_out)) 
    layer_list.append(nn.Sigmoid())
    return layer_list

## DataLoader

In [ ]:
def get_date(date):
    M, D = date.split('/')
    M = "0" + M if len(M)==1 else M
    D = "0" + D if len(D)==1 else D
    return M+D

def get_time(time):
    M, H = time.split(':')
    return M + H + "00"

def get_scenario(scenario):
    scenario = str(scenario)
    if len(scenario) == 1:
        scenario = "00" + scenario
    elif len(scenario) == 2:
        scenario = "0" + scenario
    return scenario    

In [ ]:
def process_train(base):
    original = pd.read_csv(os.path.join(base, "train.csv"))
    train = []

    for record in trange(len(original)):
        data = original.iloc[record]
        entry = {}

        entry["Date"] = get_date(data["DATE (MM/DD)"])
        entry["Time"] = get_time(data["MST"])
        entry["Global CMP22"] = data["Global CMP22 (vent/cor) [W/m^2]"]
        entry["Direct sNIP"] = data["Direct sNIP [W/m^2]"]
        entry["Azimuth Angle"] = data["Azimuth Angle [degrees]"]
        entry["Tower Dry Bulb Temperature"] = data["Tower Dry Bulb Temp [deg C]"]
        entry["Tower Wet Bulb Temperature"] = data["Tower Wet Bulb Temp [deg C]"]
        entry["Tower Dew Point Temperature"] = data["Tower Dew Point Temp [deg C]"]
        entry["Tower RH"] = data["Tower RH [%]"]
        entry["Total Cloud Cover"] = data["Total Cloud Cover [%]"]
        entry["Peak Wind Speed"] = data["Peak Wind Speed @ 6ft [m/s]"]
        entry["Avgerage Wind Direction"] = data["Avg Wind Direction @ 6ft [deg from N]"]
        entry["Station Pressure"] = data["Station Pressure [mBar]"]
        entry["Precipitation"] = data["Precipitation (Accumulated) [mm]"]
        entry["Snow Depth"] = data["Snow Depth [cm]"]
        entry["Moisture"] = data["Moisture"]
        entry["Albedo"] = data["Albedo (CMP11)"]

        #filename = os.path.join(base, "train", entry["Date"], "{0}{1}.jpg".format(entry["Date"], entry["Time"]))
        #entry['image'] = filename if os.path.isfile(filename) else None

        train.append(entry)

    return pd.DataFrame(train)

def process_test(base):
    
    test = pd.read_csv(os.path.join(base, "test.csv"))

    test = test.iloc[:,1:-1]
    test["Scenario"] = test["Scenario"].apply(get_scenario)
    test = test.sort_values(by = ['Scenario', 'Time'])
    
    return test

In [ ]:
#train = process_train(base = './')
#test = process_test(base = './')
#test.to_csv('./processed_test.csv')
train = pd.read_csv('./processed_train.csv')
test = pd.read_csv('./processed_test.csv')
tcc_idx = 7

In [ ]:
'''
train = pd.read_csv('./processed_train.csv')
train = train[train['Total Cloud Cover'] != -1]
train = train.reset_index(drop=True).iloc[:,1:]
idx_f = train[train['Total Cloud Cover'] == -7999].index
#print(len(idx_f)/3)

for x in range(int(len(idx_f)/3)):

    idx_temp = idx_f[0+x*3:3+x*3]
    train.loc[idx_temp, 'Total Cloud Cover'] = (train.loc[idx_temp[0]-5,'Total Cloud Cover'] + train.loc[idx_temp[2]+5, 'Total Cloud Cover'])/2

train.loc[train[train['Total Cloud Cover'] < 0].index, 'Total Cloud Cover'] = 0

train_inputs = []
train_targets = []
dates = train.Date.unique()

for m in dates:

    scenario = train[train.Date == m].iloc[:,2:]
    scenario = scenario.reset_index(drop=True)
    #print(scenario)
    for i in range(int((len(train[train.Date == m]) -(8*60 + 1))/10)):
        train_inputs.append(scenario.iloc[(i*10):(6*60+i*10+1),:].to_numpy(dtype=np.float32))
        train_targets.append(scenario.iloc[[i*10+6*60+x*10 for x in range(1,13)],:].to_numpy(dtype=np.float32))
    break
train_inputs = torch.from_numpy(np.asarray(train_inputs)).to(torch.float32)
train_targets = torch.from_numpy(np.asarray(train_targets)).to(torch.float32)
train_targets1 = train_targets.reshape((-1,180))
'''

"\ntrain = pd.read_csv('./processed_train.csv')\ntrain = train[train['Total Cloud Cover'] != -1]\ntrain = train.reset_index(drop=True).iloc[:,1:]\nidx_f = train[train['Total Cloud Cover'] == -7999].index\n#print(len(idx_f)/3)\n\nfor x in range(int(len(idx_f)/3)):\n\n    idx_temp = idx_f[0+x*3:3+x*3]\n    train.loc[idx_temp, 'Total Cloud Cover'] = (train.loc[idx_temp[0]-5,'Total Cloud Cover'] + train.loc[idx_temp[2]+5, 'Total Cloud Cover'])/2\n\ntrain.loc[train[train['Total Cloud Cover'] < 0].index, 'Total Cloud Cover'] = 0\n\ntrain_inputs = []\ntrain_targets = []\ndates = train.Date.unique()\n\nfor m in dates:\n\n    scenario = train[train.Date == m].iloc[:,2:]\n    scenario = scenario.reset_index(drop=True)\n    #print(scenario)\n    for i in range(int((len(train[train.Date == m]) -(8*60 + 1))/10)):\n        train_inputs.append(scenario.iloc[(i*10):(6*60+i*10+1),:].to_numpy(dtype=np.float32))\n        train_targets.append(scenario.iloc[[i*10+6*60+x*10 for x in range(1,13)],:].to_numpy

In [ ]:
def createDataset_train(path='./processed_train.csv'):

    train = pd.read_csv(path)

    train = train[train['Total Cloud Cover'] != -1]
    train = train.reset_index(drop=True).iloc[:,1:]
    idx_f = train[train['Total Cloud Cover'] == -7999].index
    #print(len(idx_f)/3)

    for x in range(int(len(idx_f)/3)):

        idx_temp = idx_f[0+x*3:3+x*3]
        train.loc[idx_temp, 'Total Cloud Cover'] = (train.loc[idx_temp[0]-5,'Total Cloud Cover'] + train.loc[idx_temp[2]+5, 'Total Cloud Cover'])/2

    train.loc[train[train['Total Cloud Cover'] < 0].index, 'Total Cloud Cover'] = 0
    
    train_inputs = []
    train_targets = []
    dates = train.Date.unique()

    for m in dates:

        scenario = train[train.Date == m].iloc[:,2:]
        scenario = scenario.reset_index(drop=True)
        scenario = scale_df(scenario)
        #print(scenario)
        for i in range(int((len(train[train.Date == m]) -(8*60 + 1))/10)):
            train_inputs.append(scenario.iloc[(i*10):(6*60+i*10+1),:].to_numpy(dtype=np.float32))
            train_targets.append(scenario.iloc[[i*10+6*60+x*10 for x in range(1,13)],:].to_numpy(dtype=np.float32))

    train_inputs = torch.from_numpy(np.asarray(train_inputs)).to(torch.float32)
    train_targets = torch.from_numpy(np.asarray(train_targets)).to(torch.float32)
    train_targets = train_targets.reshape((-1,180))

    dataset = TensorDataset(train_inputs, train_targets)
    
    return dataset 

def createDataset_test(path='./processed_test.csv'):
    
    test = pd.read_csv(path)
    idx_f = test[test['Total Cloud Cover'] == -7999].index
    
    for x in range(int(len(idx_f)/3)):
    
        idx_temp = idx_f[0+x*3:3+x*3]
        test.loc[idx_temp, 'Total Cloud Cover'] = (test.loc[idx_temp[0]-5,'Total Cloud Cover'] + test.loc[idx_temp[2]+5, 'Total Cloud Cover'])/2
    
    test.loc[test[test['Total Cloud Cover'] < 0].index, 'Total Cloud Cover'] = 0
    
    test_inputs = []
    scenarios = test.Scenario.unique()
    for scenario in scenarios:

        inputs = test.iloc[(scenario-1)*361 : (scenario-1)*361 + 361, 3:]
        scenario = scale_df(inputs)
        test_inputs.append(inputs.to_numpy(dtype=np.float32))

    test_inputs = torch.from_numpy(np.asarray(test_inputs)).to(torch.float32)
    #print(test_inputs.size())
    test_inputs = TensorDataset(test_inputs)
    
    return test_inputs
    

In [ ]:
dataset = createDataset_train()
test_ds = createDataset_test()
train_ds, val_ds = random_split(dataset, [len(dataset)-500, 500], generator = torch.Generator().manual_seed(42))

In [ ]:
for xb, yb in train_ds:
    print(xb.size(), yb.size())
    break
for (xb) in test_ds:
    print(xb[0].size())
    break
    
test_dl =  DataLoader(test_ds, batch_size = 64, shuffle=True)
for xb in (test_dl):
    print(xb[0].size())
    break

torch.Size([361, 15]) torch.Size([180])
torch.Size([361, 15])
torch.Size([64, 361, 15])


## Model

In [ ]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(EncoderLSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
    
    def forward(self, input_tensor):
        
        out, (hn, cn) = self.lstm(input_tensor)
        
        return out[:,-1,:]

In [ ]:
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_dim, num_layer):
        super(DecoderLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.device = get_device()
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, bidirectional=False, batch_first=True)
        
    def forward(self, input_tensor, time):
        lstm_outs = {}
        hidden = torch.zeros(num_layers, input_tensor.size(0), self.hidden_dim).to(self.device)
        cell = torch.zeros(num_layers, input_tensor.size(0), self.hidden_dim).to(self.device)
        for t in range(10, time+10, 10):
            input_tensor, (hn, cn) = self.lstm(input_tensor, (hidden, cell))
            lstm_outs[t] = input_tensor
        
        return lstm_outs

In [ ]:
class Linear(nn.Module):
    def __init__(self,input_dim, output_dim, layers):
        super(Linear, self).__init__()
        layer_list = construct_fnn(input_dim, output_dim, layers)
        self.fc = nn.Sequential(*layer_list)
        
    def forward(self, x):
        x = torch.flatten(x, start_dim = 1)
        out = self.fc(x)
        return out

In [ ]:
class Pipeline(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, layers):
        super(Pipeline, self).__init__()
        self.device = get_device()
        self.encoder = EncoderLSTM(input_dim, hidden_dim, num_layers)
        self.decoder = DecoderLSTM(hidden_dim, num_layers)
        self.linear = Linear(hidden_dim, output_dim, layers)
        
    def forward(self, x, time):
        
        out = self.encoder(x)
        out = out.unsqueeze(1)
        outputs = self.decoder(out, time)
        final_outputs = torch.empty((x.size(0), 0)).to(self.device)
        for i in range(10, time+10, 10):
            outputs[i] = outputs[i].squeeze(1)
            out = self.linear(outputs[i])
            final_outputs = torch.cat((final_outputs, out), dim=1)
        
        return final_outputs

In [ ]:
input_dim = 15
hidden_dim = 32
num_layers = 1
layers = [64, 128, 64]
output_dim = 15
time = 120
batch_size = 64

In [ ]:
encoder = EncoderLSTM(input_dim, hidden_dim, num_layers)
decoder = DecoderLSTM(hidden_dim, num_layers)
linear = Linear(hidden_dim, output_dim, layers)

pipeline = Pipeline(input_dim, hidden_dim, output_dim, num_layers, layers)

In [ ]:
'''
x = torch.rand((batch_size, 361, 15))
print("Encoder Input Dims : ", x.size())
out = encoder(x)
print("Encoder Output Dims : ", out.size())
out = out.unsqueeze(1)
out = decoder(out, time)
print("Decoder Output Dims : ", out[10].size())
out[10] = out[10].squeeze(1)
out[10] = linear(out[10])
print("Linear Output Dims : ", out[10].size())


print("-----------")
print("PipeLine Input Dims : ", x.size())
out = pipeline(x, time)
print("PipeLine Output Dims : ", out.size())
'''

'\nx = torch.rand((batch_size, 361, 15))\nprint("Encoder Input Dims : ", x.size())\nout = encoder(x)\nprint("Encoder Output Dims : ", out.size())\nout = out.unsqueeze(1)\nout = decoder(out, time)\nprint("Decoder Output Dims : ", out[10].size())\nout[10] = out[10].squeeze(1)\nout[10] = linear(out[10])\nprint("Linear Output Dims : ", out[10].size())\n\n\nprint("-----------")\nprint("PipeLine Input Dims : ", x.size())\nout = pipeline(x, time)\nprint("PipeLine Output Dims : ", out.size())\n'

In [ ]:
print(f"Model Params : {sum(p.numel() for p in pipeline.parameters())/1e3}K")

Model Params : 34.895K


In [ ]:
#print(out[0][0:12]) #Corresponds to weather data at 10th Minute

## Trainer

In [ ]:
for (x, y) in train_ds:
    print(x.size(), y.size())
    break

torch.Size([361, 15]) torch.Size([180])


In [ ]:
class Trainer():
    def __init__(self, train_ds, val_ds, test_ds, batch_size, epoch, lr, checkpoint):
        
        self.device = get_device()
        self.checkpoint = checkpoint
         
        self.test_ds = test_ds    
        self.train_dl, self.val_dl, self.test_dl = self.get_iterator(train_ds, val_ds, test_ds, batch_size)
        self.train_size = len(train_ds)
        self.val_size = len(val_ds)
        
        self.model = self.get_model().to(self.device)
        self.criterion = self.get_criterion().to(self.device)
        self.optimizer = self.get_optimizer(self.model, lr)
        
        self.train_loss = []
        self.val_loss = []
        
        self.start_epoch = 1
        self.end_epoch = epoch
        
    def get_iterator(self, train_ds, val_ds, test_ds, batch_size=64):
        train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle=True)
        val_dl = DataLoader(val_ds, batch_size = batch_size, shuffle=True)
        test_dl = DataLoader(test_ds, batch_size = batch_size, shuffle=False)
        
        return train_dl, val_dl, test_dl
    
    def get_criterion(self):
        return nn.L1Loss()
    
    def get_optimizer(self, model, lr):
        return torch.optim.SGD(model.parameters(), lr)
    
    def get_model(self):
        model = Pipeline(input_dim, hidden_dim, output_dim, num_layers, layers)
        return model
    
    def move_to_device(self):
        self.train_dl = DeviceDataLoader(self.train_dl, self.device)
        self.val_dl = DeviceDataLoader(self.val_dl, self.device)
        to_device(self.model, self.device);
    
    def save(self, epoch, checkpoint):
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            }, os.path.join(checkpoint, "model.pth"))
        
        torch.save({
            'epoch': epoch,
            'loss': (self.train_loss, self.val_loss), 
        }, os.path.join(checkpoint, "loss.pth"))
        
    def load(self, checkpoint):
        if os.path.exists(os.path.join(checkpoint, "model.pth")):
            checkpoints = torch.load(os.path.join(checkpoint, "model.pth"), map_location = self.device)
            self.model.load_state_dict(checkpoints['model_state_dict'])
            self.optimizer.load_state_dict(checkpoints['optimizer_state_dict'])
            
        if os.path.exists(os.path.join(checkpoint, "loss.pth")):
            checkpoints = torch.load(os.path.join(checkpoint, "loss.pth"), map_location = self.device)
            self.train_loss, self.val_loss = checkpoints['loss']
            return checkpoints['epoch']
        return 0
    
    def plot_loss(self, epoch, checkpoint):
       # assert epoch == len(self.train_loss)
        epochs = np.arange(1, len(self.train_loss)+1)
        plt.plot(epochs, self.train_loss, label="Train Loss", color="blue")
        plt.plot(epochs, self.val_loss, label="Val Loss", color="red")
        
        plt.title("Loss - " + str(epoch))
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend(loc="best")
        
        plt.savefig(os.path.join(checkpoint, "loss.png"))
        plt.close()
     
    
    def train(self):
        epoch_loss = 0
        
        #gc.collect
        torch.cuda.empty_cache()
        self.model.train()
        
        with torch.autograd.set_detect_anomaly(True):
            bar = pyprind.ProgBar(len(self.train_dl), bar_char = '█')
            for index, (inputs, targets) in enumerate(self.train_dl):
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)
                
                self.optimizer.zero_grad()
                outputs = self.model(inputs, 120)
                loss = self.criterion(outputs, targets)
                
                epoch_loss += loss.item()/len(self.train_dl)
                
                self.optimizer.step()
                
                bar.update()
                #gc.collect()
                torch.cuda.empty_cache()
            
        return epoch_loss
    
    @torch.no_grad()
    def evaluate(self):
        predicted = []
        epoch_loss = 0
        
        #gc.collect
        torch.cuda.empty_cache()
        
        self.model.eval()
        bar = pyprind.ProgBar(len(self.val_dl), bar_char='█')
        
        for idx, (inputs, targets) in enumerate(self.val_dl):
            
            inputs = inputs.to(self.device)
            targets = targets.to(self.device)
            
            outputs = self.model(inputs, 120)
            
            loss = self.criterion(outputs, targets)
            
            epoch_loss += loss.item()/len(self.val_dl)
            
            bar.update()
            #gc.collect()
            torch.cuda.empty_cache()
            
        return epoch_loss
    
    @torch.no_grad()
    def test(self):
        #gc.collect
        torch.cuda.empty_cache()
        self.model.eval()
        sample = pd.read_csv('./submission_sample.csv')
        df = sample.copy()
        for i in range(len(self.test_ds)):
            output = trainer.pred_single(self.test_ds[i][0].unsqueeze(0).to(get_device()))
            
            df.iloc[i,1] = output[0][index[2]].detach().cpu().numpy()*100
            df.iloc[i,2] = output[0][index[5]].detach().cpu().numpy()*100
            df.iloc[i,3] = output[0][index[8]].detach().cpu().numpy()*100
            df.iloc[i,4] = output[0][index[11]].detach().cpu().numpy()*100

        df.to_csv(os.path.join(self.checkpoint, 'submission.csv'), index = False)

        
    
    def fit(self,train=True, next=True, test=False):
        
        if train:
            if next:
                self.start_epoch = self.load(self.checkpoint)

            for epoch in range(self.start_epoch+1, self.end_epoch+1, 1):

                print("Starting Epoch[{0}/{1}]".format(epoch, self.end_epoch))

                epoch_loss = self.train()

                self.train_loss.append(epoch_loss)

                #time.sleep(1)
                print("Training Loss : {}".format(epoch_loss))

                epoch_loss = self.evaluate()

                self.val_loss.append(epoch_loss)
                print("Val Loss : {}".format(epoch_loss))

                self.save(epoch, self.checkpoint)
                self.plot_loss(epoch, self.checkpoint)

        if test :   
            self.test()
            
        if not(test) and not(train):
            print("What else do you want me to do? I can sing a song for you :)")
            
    def pred_single(self, input):
        self.model.eval()
        return self.model(input, 120)
    
        

## Training

*Model Versions*
- v1 : Opt : SGD, Act : Tanh, Epoch : 10, Lr : 1e-3

In [ ]:
checkpoint = './checkpoints/v1'

In [ ]:
trainer = Trainer(train_ds, val_ds, test_ds, batch_size=batch_size, epoch=10, lr=1e-3, checkpoint=checkpoint)

In [ ]:
trainer.fit(train=False, next=False, test=True)

## Predictions

In [ ]:
with torch.no_grad():
    for xb, yb in val_ds:
        
        output = trainer.pred_single(xb.unsqueeze(0).to(get_device()))
        print((yb.to(get_device()) - output).size())
        break
    

torch.Size([1, 180])


In [ ]:
tcc_idx = 7
index = [(tcc_idx + i*15) for i in range(12) ]
final_tcc = []
for idx in index:
    final_tcc.append((output[0][idx].item(),yb[idx].item()))
final_tcc

[(0.6805456280708313, 0.8865979313850403),
 (0.9140076041221619, 0.9278350472450256),
 (0.6115224361419678, 0.938144326210022),
 (0.7167075276374817, 0.9587628841400146),
 (0.7031257748603821, 0.9587628841400146),
 (0.7055176496505737, 0.8144329786300659),
 (0.7052063941955566, 0.7216494679450989),
 (0.7052353024482727, 0.8453608155250549),
 (0.7052298188209534, 0.9587628841400146),
 (0.7052300572395325, 0.9793814420700073),
 (0.7052282094955444, 0.9793814420700073),
 (0.7052288055419922, 0.9793814420700073)]

## Playground